![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_XlmRoBertaSentenceEmbeddings.ipynb)

# Import ONNX XlmRoBertaSentenceEmbeddings models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for XlmRoBertaSentenceEmbeddings from HuggingFace and they have to be in `Fill Mask` category. Meaning, you cannot use XlmRoBertaSentenceEmbeddings models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.34.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers[onnx]==4.52.3 optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [xlm-roberta-base](https://huggingface.co/xlm-roberta-base) model from HuggingFace as an example and load it as a `ORTModelForFeatureExtraction`, representing an ONNX model.
- In addition to the XLM-RoBERTa model, we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import XLMRobertaTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction

MODEL_NAME = "xlm-roberta-base"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The model xlm-roberta-base was already converted to ONNX but got `export=True`, the model will be converted to ONNX once again. Don't forget to save the resulting model with `.save_pretrained()`


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

('onnx_models/xlm-roberta-base/tokenizer_config.json',
 'onnx_models/xlm-roberta-base/special_tokens_map.json',
 'onnx_models/xlm-roberta-base/sentencepiece.bpe.model',
 'onnx_models/xlm-roberta-base/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 1089044
-rw-r--r-- 1 root root        640 Jun 27 03:58 config.json
-rw-r--r-- 1 root root 1110092472 Jun 27 03:58 model.onnx
-rw-r--r-- 1 root root    5069051 Jun 27 03:58 sentencepiece.bpe.model
-rw-r--r-- 1 root root        280 Jun 27 03:58 special_tokens_map.json
-rw-r--r-- 1 root root       1203 Jun 27 03:58 tokenizer_config.json


We need to move `sentencepiece.bpe.model` from the tokenizer to `assets` folder which Spark NLP will look for

In [4]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/sentencepiece.bpe.model {EXPORT_PATH}/assets

## Import and Save XlmRoBertaSentenceEmbeddings in Spark NLP

- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [5]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 36.5 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [6]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `XlmRoBertaSentenceEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `XlmRoBertaSentenceEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [7]:
from sparknlp.annotator import XlmRoBertaSentenceEmbeddings

bert = XlmRoBertaSentenceEmbeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["document"])\
    .setOutputCol("bert")\
    .setCaseSensitive(True)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [8]:
bert.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [9]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX XlmRoBertaSentenceEmbeddings model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [10]:
! ls -l {MODEL_NAME}_spark_nlp

total 1089204
drwxr-xr-x 2 root root       4096 Jun 27 04:03 metadata
-rw-r--r-- 1 root root 1110261996 Jun 27 04:04 xlmroberta_sentence_onnx
-rw-r--r-- 1 root root    5069051 Jun 27 04:03 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XlmRoBertaSentenceEmbeddings model 😊

In [14]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

bert_loaded = XlmRoBertaSentenceEmbeddings.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document"]) \
    .setOutputCol("bert")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    bert_loaded
])

data = spark.createDataFrame([[
    "William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist."
]], ["text"])

model = pipeline.fit(data)
result = model.transform(data)

result.selectExpr("explode(bert.embeddings[0]) as embedding").show()

+-------------+
|    embedding|
+-------------+
| 4.0072636E-4|
|   0.00312963|
| 0.0013518634|
|-0.0014537025|
|-0.0015880099|
| -0.009523226|
| 0.0016372511|
|  0.006321915|
|-1.6188821E-4|
| -0.008094354|
|  0.005312449|
| 0.0035219616|
|  0.007570335|
| -2.236374E-4|
|  0.001648069|
|-0.0011625341|
| 0.0012606808|
|-0.0021722228|
| 0.0032363958|
| 0.0029782664|
+-------------+
only showing top 20 rows



That's it! You can now go wild and use hundreds of XlmRoBertaSentenceEmbeddings models from HuggingFace 🤗 in Spark NLP 🚀
